# Background



In this exercise, we implement the concept of structural stochastic volatility which derives from different noise levels in the demand of fundamentalist and chartists and time varying market share of the two groups. 

We consider the || name here || approach of endogenous switching between the trading strategies and estimate the model by method of simulated moments. where choice of moments reflects the basic stylized facts of daily returns of a stock market index. 

In [1]:
%load_ext Cython

In [2]:
%matplotlib inline



In [ ]:
%%cython 

import numpy as np
cimport numpy as np
import time
import argparse
import pandas as pd
import seaborn as sns
from statsmodels.tsa.stattools import acf
from yahoofinancials import YahooFinancials
import scipy.optimize as opt
from arch import arch_model
from scipy.stats import kurtosis, skew
import matplotlib as mpl
import statsmodels.api as sm
import matplotlib.pyplot as plt
import sys
sns.set()
import os, pickle as pkl
plt.rcParams['figure.figsize'] = (15, 6)
from itertools import product
from itertools import chain
import random

LAG_1_AUTOCORRELATION_RAW = 'Lag-1 Autocorrelation (raw)'
MEAN_ABS = 'Mean (absolute)' 
HILL_ESTIMATE = 'Hill estimator (absolute)' 
LAG_1_AUTOCORRELATION_ABS = 'Lag-1 Autocorrelation (absolute)'
LAG_5_AUTOCORRELATION_ABS = 'Lag-5 Autocorrelation (absolute)'
LAG_10_AUTOCORRELATION_ABS = 'Lag-10 Autocorrelation (absolute)'
LAG_25_AUTOCORRELATION_ABS = 'Lag-25 Autocorrelation (absolute)'
LAG_50_AUTOCORRELATION_ABS = 'Lag-50 Autocorrelation (absolute)'
LAG_100_AUTOCORRELATION_ABS = 'Lag-100 Autocorrelation (absolute)'

class DataLoader:
    def __init__(self, assets, start_date, end_date, frequency='daily'):
        self.assets = assets
        self.start_date = start_date
        self.end_date = end_date
        self.frequency = frequency
        
    
    def get_empirical_data(self, log_returns = True, commentary=True):
        yahoo_financial = YahooFinancials(self.assets)
        try:
            data = yahoo_financial.get_historical_price_data(start_date=self.start_date,
                                                             end_date=self.end_date,
                                                             time_interval=self.frequency)
            df = pd.DataFrame({
            a: {x['formatted_date']: x['adjclose'] for x in data[a]['prices']} for a in assets
        })
        except:
            print("Not able to download data using yahoo financials. System will exit here.")
            sys.exit(0)
            
        df = df.set_index(pd.DatetimeIndex(df.index))
        
        if log_returns:
            df = np.log(df / df.shift(1)).dropna()
            
        if commentary:
            print("Data extraction successful....\nSummary: \n\n", df.describe())
            
        return df
    


class Moments:
    @staticmethod
    def get_numerical_moments(series, moment_list, averaging_offset=None):

        cdef int moment_max
        if averaging_offset is None:
            moment_max = np.max(moment_list)
        else:
            moment_max = np.max(moment_list) + averaging_offset
        
        results = []
        
        if averaging_offset is None:
            results = [np.corrcoef(series[:-moment], series[moment:])[0,1] for moment in moment_list]
        else:
            results = [np.mean([np.corrcoef(series[:-mom], series[mom:])[0,1] for mom in range(1,averaging_offset+2)]) if moment==1 \
                       else np.mean([np.corrcoef(series[:-mom], series[mom:])[0,1] for mom in range(moment-averaging_offset,moment + averaging_offset+1)])\
                      for moment in moment_list]

    
        return results
    @staticmethod
    def get_hill_estimator(series, k=0.05):
        abs_series = np.sort(np.abs(series))
        num_terms = int(k*len(abs_series))
        gamma = np.mean(np.log(abs_series[-num_terms:])) - np.log(abs_series[-(num_terms+1)])
        return 1.0/gamma
    
    
class HPM:
    
    HPM_INIT = {
                    "mu": 0.01, 
                    "phi": 0.12,
                    "chi": 1.5,
                    "pstar": 100, 
                    "sigma_f": 0.758,
                    "sigma_c": 2.087 
                }
    
    
    def optimise_criterion(self, params_dict_values, actual_ts_moments, weighing_matrix, params_dict_keys, price0, ts_length, buffer_multiplier=5):
        
        # reconstruct params dict for simulation
        params_dict = {params_dict_keys[i]:params_dict_values[i] for i in range(len(params_dict_keys))}

        
        # Calculating simulated price
        simulated_price = self.simulate(param_dict = params_dict, 
                                        num_paths = 1, 
                                        length=buffer_multiplier*ts_length, 
                                        price0=price0)
        # Ignore first few points
        simulated_price = simulated_price[ts_length:, :]
        
        # Simulated price returns
        simulated_price_returns = np.log(simulated_price[1:, :]) - np.log(simulated_price[:-1, :])
        
        # Simulated time series moments 
        simulated_ts_moments, _ = self.get_moments(simulated_price_returns[:, 0])
        
        # Error between simulated and actual moments 
        error = np.array(actual_ts_moments).reshape(1, -1) - np.array(simulated_ts_moments).reshape(1, -1)
        
        return (error @ weighing_matrix @ error.T).flatten()[0]
    
    def calibrate(self, initial_params_dict, ts, weighing_matrix, price0, n_max_iter=1000, grid_init_search=True):
        # Separate keys and values 
        keys, values = list(initial_params_dict.keys()), tuple(initial_params_dict.values())
        
        # Minimize the weighted error
        empirical_moments, empirical_headers = self.get_moments(ts)
        
        if grid_init_search:
            print("\nSearching for optimised starting values....")
            minima = np.inf
            minima_dict = None
            for dictionary in self.get_shocked_dictionaries(initial_params_dict):
                keys_dummy, values_dummy = list(dictionary.keys()), tuple(dictionary.values())
                error_value = self.optimise_criterion(values_dummy, empirical_moments, weighing_matrix, keys_dummy, price0, len(ts))
                if error_value <= minima:
                    print("Reduced error from ", minima, "to", error_value)
                    minima_dict = dictionary.copy()
                    minima = error_value
            
            print("Initial dict: ", initial_params_dict)
            initial_params_dict = minima_dict.copy()
            print("Updated dict: ", initial_params_dict)
            
            # Separate keys and values 
            keys, values = list(initial_params_dict.keys()), tuple(initial_params_dict.values())

        
        
        calibrated_result = opt.minimize(self.optimise_criterion, values, args=(empirical_moments, weighing_matrix, keys, price0, len(ts)),\
                                        method='Nelder-Mead', options={'maxiter': n_max_iter, 'disp': True})
        
        # Calibrated parameters
        calibrated_params = {keys[i]:calibrated_result.x[i] for i in range(len(keys))}
        
        return calibrated_params, initial_params_dict
    
    
    def get_shocked_dictionaries(self, init_params_dict, shock=0.2, num_points=3):
        keys, values = list(init_params_dict.keys()), list(init_params_dict.values())
        values_modified = list(product(*[list(np.linspace((1-shock)*val, (1+shock)*val, num_points)) for val in values]))
        for val in values_modified:
            yield {keys[i]: val[i] for i in range(len(keys))}
            
    
    
    def get_moments(self, ts, commentary=False):
        '''
        Returns the moments in the following order:
        ['Lag 1 autocorrelation from raw returns', 'Mean absolute level', 'Lag 1 autocorrelation from abs returns',
        'Lag 5 autocorrelation from abs returns', 'Lag 10 autocorrelation from abs returns', 
        'Lag 25 autocorrelation from abs returns', 'Lag 50 autocorrelation from abs returns',
        'Lag 100 autocorrelation from abs returns', 'Hill estimator']
        '''
        if commentary:
            print("ts :", ts)
        
        m = []
        
        abs_ts = np.abs(ts)
        
        
        # First order autocorrelation coefficient from raw returns

        m = m + Moments.get_numerical_moments(ts, [1])
        
        # Mean of absolute returns 
        m = m + [np.mean(abs_ts)]
        
        
        # Hill estimator 
        m = m + [Moments.get_hill_estimator(abs_ts)]

        
        # lag [1,5] from absolute returns 
        m = m + Moments.get_numerical_moments(abs_ts, [1, 5, 10, 25, 50, 100], averaging_offset=1)
        
        return m, [LAG_1_AUTOCORRELATION_RAW, MEAN_ABS, HILL_ESTIMATE, 
                   LAG_1_AUTOCORRELATION_ABS, LAG_5_AUTOCORRELATION_ABS, 
                   LAG_10_AUTOCORRELATION_ABS, LAG_25_AUTOCORRELATION_ABS, 
                  LAG_50_AUTOCORRELATION_ABS, LAG_100_AUTOCORRELATION_ABS]
    
        
    def get_weighing_matrix(self, actual_ts, num_bootstraps=5000,  method = 'block-bootstrap'):
        
        if method.lower() =='block-bootstrap':
            print("Actual ts :", actual_ts)
            
            moment_samples = np.zeros((9, num_bootstraps))
            abs_ts = np.abs(actual_ts)
            
            num_blocks_250 = len(actual_ts) // 250
            num_blocks_750 = len(actual_ts) // 750
            
            blocks_250 = [actual_ts[j*250:(j+1)*250] for j in range(num_blocks_250)]
            blocks_750 = [actual_ts[j*750:(j+1)*750] for j in range(num_blocks_750)]
            
            for i in range(num_bootstraps):
                
                m = []
                
                modified_ts = list(chain.from_iterable(random.choices(blocks_250, k=num_blocks_250)))
                
                m = self.get_moments(modified_ts)[0][:-4]
        
                modified_abs_ts = list(chain.from_iterable(random.choices(blocks_750, k=num_blocks_750)))
            
                m = m + self.get_moments(modified_abs_ts)[0][-4:]
                
                moment_samples[:, i] = m
                
            weighing_matrix = np.linalg.inv(np.cov(moment_samples))
            
        elif method.lower()=='equal':
            weighing_matrix = np.eye(9)
            
        return weighing_matrix
    
        
    
    def simulate(self, param_dict, num_paths, price0, length,  seed=1001):
        
        np.random.seed(seed)
        
        cdef np.ndarray price = np.zeros((length+1, num_paths), dtype=np.float)
        cdef np.ndarray demand_fundamentalist = np.zeros((length+1, num_paths), dtype=np.float)
        cdef np.ndarray demand_chartist = np.zeros((length+1, num_paths), dtype=np.float)
        cdef np.ndarray num_chartist = np.ones((length+1, num_paths), dtype=np.float )*0.5
        cdef np.ndarray num_fundamentalist = np.ones((length+1, num_paths), dtype=np.float)*0.5
        
        price[0, :] = np.array([price0]*num_paths)

        for t in range(1, length+1):
            price[t, :] = price[t-1, :] + param_dict['mu']*(num_fundamentalist[t-1, :]*demand_fundamentalist[t-1, :] \
                                                     + num_chartist[t-1, :]*demand_chartist[t-1, :])
            
            demand_fundamentalist[t, :] = param_dict['phi']*(param_dict['pstar'] - price[t,:]) \
                                        + np.random.normal(scale= param_dict['sigma_f'], size=num_paths)
            demand_chartist[t, :] = param_dict['chi']*(price[t]-price[t-1]) \
                                        + np.random.normal(scale= param_dict['sigma_c'], size=num_paths)
             
        return price
    
    
    
def yield_slices(series, slice_length, min_size=200):
    slice_index = 0
    while True:
        sliced_series = series[slice_length*slice_index:slice_length*(slice_index+1)]
        if len(sliced_series) >= min_size: 
            print(sliced_series)
            yield sliced_series
        slice_index +=1
        
def calculate_series_return(series, log=False):
#     print(series[1:])
    if log:
        return (np.log(series) - np.log(series.shift(1))).dropna()
    else:
        return (series - series.shift(1)).dropna()
#         return np.subtract(series[1:], series[:-1])


    
def write_timer():
    elapsed_time = None
    
    while True:
        if elapsed_time is None:
            elapsed_time = 0
            last_time = time.time()
            yield ""
        else:
            elapsed_time = time.time() - last_time
            last_time = time.time()
            yield elapsed_time
            
            
            

assets = ['^GSPC']
assets_dict = {'^GSPC':'SPX'}
start_date = '2015-12-31'
end_date = '2020-07-29'
frequency = 'daily'
parent_directory = start_date + " to " + end_date + " (without initial grid search)"
if not os.path.isdir(parent_directory):
    os.mkdir(parent_directory)
    

filename = os.path.join(parent_directory, ("_".join([assets_dict[asset] for asset in assets_dict])) + start_date + "--" + end_date + " (" + frequency + ").xlsx" )   

if os.path.isfile(filename):
    print("Reading data from file")
    df = pd.read_excel(filename, index_col=0, parse_dates=True)
else:
    dl = DataLoader(assets, start_date, end_date, frequency=frequency)
    df = dl.get_empirical_data(log_returns=False)
    df.index.name='Date'
    df.to_excel(filename)
            
num_paths = 5000
slice_length = 500
min_slice_length = 250 # In case not too many points are left
n_optimizer_iterations = 1000
moments_simulated_df_list = []
calibrated_dicts_list = []
initial_dicts_list = []

for asset in assets:
    asset_log_price_ts = np.log(df[asset])

    
    hpm_obj = HPM()
    slice_index = 0
        
    while True:
        
        price_ts = asset_log_price_ts[slice_length*slice_index:slice_length*(slice_index+1)]
        slice_index +=1
        
        if len(price_ts) <= min_slice_length: 
            break 
        print("*"*5, "Running for time period :", price_ts.index[0].strftime("%Y-%m-%d") + " - " + price_ts.index[-1].strftime("%Y-%m-%d"), "*"*5)
        
        return_ts = calculate_series_return(price_ts)
        
        timer = write_timer(); next(timer)
    
        weighing_matrix = hpm_obj.get_weighing_matrix(return_ts, method='equal')
        print("Time taken to retrive weighing matrix", next(timer))
        
        init_params_dict = HPM.HPM_INIT
        init_params_dict['pstar'] = np.mean(return_ts)
        
        calibrated_params_dict, initial_params_dict_grid_search  = hpm_obj.calibrate(initial_params_dict = init_params_dict, 
                                                                          ts = return_ts.values, 
                                                                          weighing_matrix = weighing_matrix,
                                                                          price0= price_ts[0],
                                                                          n_max_iter = n_optimizer_iterations, 
                                                                          grid_init_search=True)
        
        
        print("Time taken for calibration :", next(timer))
        price_simulated = hpm_obj.simulate(calibrated_params_dict,
                                           num_paths=num_paths,
                                           price0= price_ts[0],
                                           length=5*len(asset_log_price_ts))
        
        print("Time taken for simulation :", next(timer))
        price_simulated = price_simulated[len(price_ts):, :]
        
        returns_simulated = price_simulated[1:, :] - price_simulated[:-1, :]

        moments_simulated = [hpm_obj.get_moments(returns_simulated[:, i])[0] for i in range(num_paths)]
        
        moments_actual, headers = hpm_obj.get_moments(return_ts, commentary=False)
        
        moments_simulated_df = pd.DataFrame(np.array(moments_simulated), columns=headers)
        
        print("Time taken to get simulated moments :", next(timer))
        for header in headers:
            fig, ax = plt.subplots()
            ax.hist(moments_simulated_df[header], density=True, bins=50, color='purple')
            ax.axvline(moments_actual[headers.index(header)], color='red', linewidth=2)
            plt.title(header + " (Datapoints: " + str(len(price_ts)) + ", Daterange: " + price_ts.index[0].strftime("%Y-%m-%d") + "-" + price_ts.index[-1].strftime("%Y-%m-%d") + ")")
            if not os.path.isdir(os.path.join(parent_directory, header)):
                os.mkdir(os.path.join(parent_directory, header))
            plt.savefig(os.path.join(parent_directory, header, price_ts.index[0].strftime("%Y-%m-%d") + "-" + price_ts.index[-1].strftime("%Y-%m-%d") + '.jpg'), format='jpeg')
            plt.show()
            
        moments_simulated_df['Start Date'] = price_ts.index[0]
        moments_simulated_df['End Date'] = price_ts.index[-1]
        
        for header in headers:
            moments_simulated_df[header + "_Empirical"] = moments_actual[headers.index(header)]
                                                                         
        moments_simulated_df_list.append(moments_simulated_df)
        
        calibrated_dicts_list.append(calibrated_params_dict)
        initial_dicts_list.append(initial_params_dict_grid_search)
        
        
    moments_simulated_combined = pd.concat(moments_simulated_df_list)


    for header in headers:
        fig, ax = plt.subplots()
        ax.hist(moments_simulated_df[header], density=True, bins=100, color='purple')
        ax.hist(moments_simulated_df[header + "_Empirical"], density=True, bins=20, color='red')
        plt.title(header + " : Simulated vs Empirical")
        plt.savefig(os.path.join(parent_directory, header + " - Simulated vs Empirical" + '.jpg'), format='jpeg')
        plt.show()
        
    
    calibrated_dicts_df = pd.DataFrame(calibrated_dicts_list)
    initial_dicts_df = pd.DataFrame(initial_dicts_list)
    
    initial_dicts_df.rename(lambda x: x+ "_Initial", axis='columns')
    
    combined_dicts_df = pd.concat([calibrated_dicts_df, initial_dicts_df], axis=1)
    
    combined_dicts_df.to_excel(os.path.join(parent_directory, key + " - Initial vs Final.xlsx"))
    
    for key in calibrated_params_dict.keys():
        fig, ax = plt.subplots()
        ax.scatter(combined_dicts_df.index, combined_dicts_df[key], color='purple', label="Final")
        ax.scatter(combined_dicts_df.index, combined_dicts_df[key + "_Initial"], color='red', label="Initial")
        ax.legend()
        plt.title(key + " : Initial vs Final")
        plt.savefig(os.path.join(parent_directory, key + " - Initial vs Final.jpg"), format='jpeg')
        plt.show()
        
        



Data extraction successful....
Summary: 

              ^GSPC
count  1151.000000
mean   2609.063177
std     361.213298
min    1829.079956
25%    2341.890015
50%    2663.419922
75%    2879.630005
max    3386.149902
***** Running for time period : 2015-12-31 - 2017-12-22 *****
Time taken to retrive weighing matrix 2.5987625122070312e-05

Searching for optimised starting values....
Reduced error from  inf to 1.1630556975639659
Reduced error from  1.1630556975639659 to 0.7057809974826492
Reduced error from  0.7057809974826492 to 0.6970286786923612
Reduced error from  0.6970286786923612 to 0.6969098199506797
Reduced error from  0.6969098199506797 to 0.6967909813993147
Reduced error from  0.6967909813993147 to 0.6096546226323797
Reduced error from  0.6096546226323797 to 0.4720393342612843
Reduced error from  0.4720393342612843 to 0.36662777120899803
Reduced error from  0.36662777120899803 to 0.35614544702588596
Reduced error from  0.35614544702588596 to 0.345896085001731
